In [13]:
import utils as u
import plotly.graph_objects as go
import pandas as pd

In [14]:
X = 12345
Y = 6789

In [15]:
def naive_mult(x: int, y: int):
    x, y = min(x, y), max(x, y)
    result = 0
    for _ in range(x):
        result += y
    return result


@u.timed
def naive_mult_timed(x: int, y: int):
    return naive_mult(x, y)

In [16]:
naive_mult_timed(X, Y)

Elapsed time: 319000.0000 nanoseconds


83810205

In [17]:
def karatsuba_mult(x: int, y: int):
    # Base case for recursion: when num1 or num2 are small enough
    if x < 10 or y < 10:
        return x * y

    # Calculate the size of the numbers
    m = max(len(str(x)), len(str(y))) // 2

    # Split the numbers
    high1 = x // 10**m
    low1 = x % 10**m
    high2 = y // 10**m
    low2 = y % 10**m

    # Recursive steps
    z0 = karatsuba_mult(low1, low2)
    z1 = karatsuba_mult(low1 + high1, low2 + high2)
    z2 = karatsuba_mult(high1, high2)

    # Karatsuba formula
    return (z2 * 10 ** (m * 2)) + ((z1 - z2 - z0) * 10**m) + z0


@u.timed
def karatsuba__mult_timed(x: int, y: int):
    return karatsuba_mult(x, y)

In [18]:
karatsuba__mult_timed(X, Y)

Elapsed time: 17000.0000 nanoseconds


83810205

In [19]:
pairs_count = 100
min_int = 100000
max_int = 200000

pairs = u.generate_sorted_number_pairs(pairs_count, min_int, max_int)
naive_times = u.sample_int_mult_algorithm(naive_mult, pairs)
karatsuba_times = u.sample_int_mult_algorithm(karatsuba_mult, pairs)

In [29]:
df = pd.DataFrame(pairs[::10], columns=["x", "y"])
df["naive (ns)"] = naive_times[::10]
df["karatsuba (ns)"] = karatsuba_times[::10]
df

,x,y,naive (ns),karatsuba (ns)
0,100081,102134,5921000,18000
1,104257,119729,5247000,9000
2,117941,129585,5143000,9000
3,127446,136719,5673000,11000
4,139437,147146,6100000,9000
5,150208,158979,6626000,9000
6,161603,165935,7163000,7000
7,166986,175760,7292000,10000
8,178711,182384,7994000,9000
9,187997,192370,8415000,10000


In [9]:
fig = go.Figure()
y_labels = [f"Call {i+1}" for i in range(pairs_count)]
fig.add_trace(
    go.Scatter(
        y=naive_times,
        mode="lines+markers",
        name="Naive Multiplication",
        line=dict(color="blue"),
    )
)
fig.add_trace(
    go.Scatter(
        y=karatsuba_times,
        mode="lines+markers",
        name="Karatsuba Multiplication",
        line=dict(color="red"),
    )
)
fig.update_layout(
    title="Elapsed Time Comparison",
    yaxis_title="Elapsed Time (nanoseconds)",
    yaxis_tickmode="array",
    yaxis_tickvals=list(range(len(y_labels))),
    yaxis_ticktext=y_labels,
    showlegend=True,
)
fig.show()